# Summarizing Text with Caurtion corpus


To build our model we will use a two-layered bidirectional RNN with LSTMs on the input data and two layers, each with an LSTM using bahdanau attention on the target data. [Jaemin Cho's tutorial](https://github.com/j-min/tf_tutorial_plus/tree/master/RNN_seq2seq/contrib_seq2seq) for seq2seq was really helpful to get the code in working order because this is my first project with TensorFlow 1.1; some of the functions are very different from 1.0. The architecture for this model is similar to Xin Pan's and Peter Liu's, here's their [GitHub page.](https://github.com/tensorflow/models/tree/master/textsum)

The sections of this project are:
- Inspecting the Data
- Preparing the Data
- Building the Model
- Training the Model
- Making Our Own Summaries

Google connect to acess you the data stored on your drive

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.22-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.22-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
#need to install older version of tf
!pip install tensorflow==1.15     

     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 3.8MB 47.1MB/s 
     |████████████████████████████████| 512kB 56.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=6ff1531b474cc3da5396b83bc45cf16a022d30fcea1e2b5286f21a0b4f18a1ca
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: 

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.15.0


### Google Connect

## Insepcting the Data

In [ ]:
news = pd.read_csv("/content/drive/Datafile.csv")

In [ ]:
news.shape

(40000, 3)

In [ ]:
news.head(10)

,url,Summary,Text
0,https://www.independent.co.uk/news/world/europ...,Non-stunned halal and kosher meat not saleable...,Animals must be stunned prior to being killed ...
1,https://techxplore.com/news/2018-07-bacteria-p...,Solar cells powered by hybrid E. coli convert ...,Researchers in Canada have developed an innova...
2,http://www.themalaymailonline.com/malaysia/art...,Southeast Asia unprepared for ISIS attacks say...,Southeast Asia is unprepared for the rapidly-r...
3,http://www.scotsman.com/business/companies/ret...,Speedy Hire rejects calls for merger with HSS,In addition to calls from Toscafund to oust it...
4,http://www.wsj.com/articles/transferwise-plans...,TransferWise to launch China services\r\n,TransferWise has announced plans to launch a c...
5,http://www.standard.co.uk/news/uk/londoners-at...,Pollution and other environmental factor linke...,"A lack of Vitamin D, polluted air, exposure to..."
6,http://www.vanitatis.elconfidencial.com/estilo...,Soft Silhouette is used to treat sagging skin,Soft Silhouette can be used to treat sagging j...
7,http://www.japantimes.co.jp/news/2016/05/02/re...,Japan's new law bans bias against disabilities,"On 1 April 2016, a new Japanese law came into ..."
8,https://www.msn.com/en-ph/foodanddrink/foodnew...,San Francisco restaurants serve up more co-wor...,"Spacious, which offers a subscription-based se..."
9,http://www.communitycare.co.uk/2017/07/20/coun...,UK councils tripled overseas hires of social ...,Difficulty attracting experienced social worke...


In [ ]:
#check columns
column_names = news.columns
print(column_names)


Index(['url', 'Summary', 'Text'], dtype='object')


In [ ]:
# Check for any nulls values
news.isnull().sum()
news.dropna(how='all')


,url,Summary,Text
0,https://www.independent.co.uk/news/world/europ...,Non-stunned halal and kosher meat not saleable...,Animals must be stunned prior to being killed ...
1,https://techxplore.com/news/2018-07-bacteria-p...,Solar cells powered by hybrid E. coli convert ...,Researchers in Canada have developed an innova...
2,http://www.themalaymailonline.com/malaysia/art...,Southeast Asia unprepared for ISIS attacks say...,Southeast Asia is unprepared for the rapidly-r...
3,http://www.scotsman.com/business/companies/ret...,Speedy Hire rejects calls for merger with HSS,In addition to calls from Toscafund to oust it...
4,http://www.wsj.com/articles/transferwise-plans...,TransferWise to launch China services\r\n,TransferWise has announced plans to launch a c...
...,...,...,...
39995,https://www.irishtimes.com/business/hammond-ta...,UK chancellor to target offshore gambling comp...,UK Chancellor Philip Hammond is expected to in...
39996,http://www.tmcnet.com/usubmit/2015/07/30/82257...,ALLOY Health platform chosen by Livongo Health...,Livongo Health empowers people who have chroni...
39997,https://chemicalwatch.com/65818/furniture-flam...,Furniture flame retardant smoke toxicity to be...,The capacity for flame retardants (FRs) to pro...
39998,https://www.insurancebusinessmag.com/asia/news...,AXA launches solutions integration platform fo...,"AXA Insurance has launched a one-stop, self-se..."


In [ ]:
# Remove null values and unneeded features

news = news.dropna()
news = news.drop(['url'], axis=1)
news = news.reset_index(drop=True)
news.head()

# drop row, if values in Summary is missing. 
news.dropna(subset=['Summary'],inplace = True)
news.dropna(subset=['Text'],inplace = True)


In [ ]:
# Inspecting some of the news
for i in range(5):
    print("news #",i+1)
    print("  the text is  :",news.Text[i])
    print("-------------")
    print("the summary is :",news.Summary[i])
    print()

news # 1
  the text is  : Animals must be stunned prior to being killed in order for halal and kosher meat to be marketed with the official European Union (EU) organic label, the European Court of Justice has ruled. The judges argued that organic labelling reflects the highest animal welfare standards, and that stunning is integral to these as it significantly reduces suffering. The UK’s Food Standards Agency estimates that 88% of animals killed under halal methods are stunned. However, a minority of halal – and all kosher – meat is produced without stunning. The case was brought by a French animal welfare association in 2012.

-------------
the summary is : Non-stunned halal and kosher meat not saleable as organic: ECJ

news # 2
  the text is  : Researchers in Canada have developed an innovative solar cell which uses bacteria to convert light into energy. The team at the University of British Columbia developed a way of genetically engineering E.coli to produce large amounts of lycope

## Preparing the Data

In [ ]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

We will remove the stopwords from the texts because they do not provide much use for training our model. However, we will keep them for our summaries so that they sound more like natural phrases. 

In [ ]:
import nltk
nltk.download('stopwords')
  
# Clean the summaries and texts
clean_summaries = []
for summary in news.Summary:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in news.Text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Summaries are complete.
Texts are complete.


In [ ]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(5):
    print("Clean news #",i+1)
    print("text  : ",clean_texts[i])
    print("summry: ",clean_summaries[i])
    print()

Clean news # 1
text  :  animals must stunned prior killed order halal kosher meat marketed official european union eu organic label european court justice ruled judges argued organic labelling reflects highest animal welfare standards stunning integral significantly reduces suffering uk’s food standards agency estimates 88 animals killed halal methods stunned however minority halal – kosher – meat produced without stunning case brought french animal welfare association 2012
summry:  non stunned halal and kosher meat not saleable as organic  ecj

Clean news # 2
text  :  researchers canada developed innovative solar cell uses bacteria convert light energy team university british columbia developed way genetically engineering e coli produce large amounts lycopene natural dye bacteria use photosynthesis coating bacteria semiconducting substance incorporating battery cell able achieve current density 0 686 milliamps per square centimetre say highest yet achieved biogenic solar cell
summry: 

In [ ]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [ ]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 80991


In [ ]:
# Load Conceptnet Numberbatch's (CN) embeddings, similar to GloVe, but probably better 
# (https://github.com/commonsense/conceptnet-numberbatch)
embeddings_index = {}
with open('/content/drive/numberbatch-en.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 516783


In [ ]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 948
Percent of words that are missing from vocabulary: 1.17%


I use a threshold of 20, so that words not in CN can be added to our 

1.   List item

1.   List item

1.   List item
2.   List item


2.   List item


2.   List item

word_embedding_matrix, but they need to be common enough in the reviews so that the model can understand their meaning.

In [ ]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 80991
Number of words we will use: 42699
Percent of words we will use: 52.72%


In [ ]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

42699


In [ ]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [ ]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 2512789
Total number of UNKs in headlines: 85158
Percent of words that are UNK: 3.39%


In [ ]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [ ]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
             counts
count  39986.000000
mean       9.174261
std        1.975717
min        1.000000
25%        8.000000
50%        9.000000
75%       10.000000
max       35.000000

Texts:
             counts
count  39986.000000
mean      54.667459
std       17.393107
min        1.000000
25%       45.000000
50%       55.000000
75%       63.000000
max      537.000000


In [ ]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

70.0
77.0
103.0


In [ ]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

11.0
12.0
14.0


In [ ]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [ ]:
# takes a long time  , this is normal

# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 70
max_summary_length = 14
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

17278
17278


## Building the Model

In [ ]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [ ]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [ ]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [ ]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_decoder

In [ ]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_decoder

In [ ]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    #initial_state = tf.contrib.seq2seq.AttentionWrapperState(enc_state[0],
    #                                                                _zero_state_tensors(rnn_size, 
    #                                                                                    batch_size, 
    #                                                                                    tf.float32)) 
    initial_state = dec_cell.zero_state(batch_size=batch_size,dtype=tf.float32).clone(cell_state=enc_state[0])

    with tf.variable_scope("decode"):
        training_decoder = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
        
        training_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_decoder = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)
        
        inference_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)

    return training_logits, inference_logits

In [ ]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [ ]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [ ]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [ ]:
# Set the Hyperparameters
epochs = 100
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

In [ ]:

# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Gr

## Training the Model

Since I am training this model on my MacBook Pro, it would take me days if I used the whole dataset. For this reason, I am only going to use a subset of the data, so that I can train it over night. Normally I use [FloydHub's](https://www.floydhub.com/) services for my GPU needs, but it would take quite a bit of time to upload the dataset and ConceptNet Numberbatch, so I'm not going to bother with that for this project.

I chose not use use the start of the subset because I didn't want to make it too easy for my model. The texts that I am using are closer to the median lengths; I thought this would be more fair.

In [ ]:
# Subset the data for training
start = 100
end = start + 100000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 16
The longest text length: 69


In [ ]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 6 #3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

  
tf.reset_default_graph()
checkpoint = "/content/drive/Model 1/Model 300/best_model.ckpt"  #300k sentence
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    # loader = tf.train.import_meta_graph(checkpoint + '.meta')
    # loader.restore(sess, checkpoint)
    #sess.run(tf.local_variables_initializer())

    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0
                
                #saver = tf.train.Saver() 
                #saver.save(sess, checkpoint)
                
            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
              
                  
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Epoch   1/100 Batch   20/268 - Loss:  7.356, Seconds: 3.12
Epoch   1/100 Batch   40/268 - Loss:  5.572, Seconds: 3.13
Epoch   1/100 Batch   60/268 - Loss:  5.526, Seconds: 3.07
Epoch   1/100 Batch   80/268 - Loss:  5.412, Seconds: 3.76
Average loss for this update: 5.92
New Record!
Epoch   1/100 Batch  100/268 - Loss:  5.484, Seconds: 3.63
Epoch   1/100 Batch  120/268 - Loss:  5.563, Seconds: 3.86
Epoch   1/100 Batch  140/268 - Loss:  5.367, Seconds: 3.59
Epoch   1/100 Batch  160/268 - Loss:  5.550, Seconds: 3.77
Average loss for this update: 5.467
New Record!
Epoch   1/100 Batch  180/268 - Loss:  5.317, Seconds: 3.99
Epoch   1/100 Batch  200/268 - Loss:  5.165, Seconds: 4.38
Epoch   1/100 Batch  220/268 - Loss:  5.191, Seconds: 4.22
Epoch   1/100 Batch  240/268 - Loss:  5.293, Seconds: 4.45
Epoch   1/100 Batch  260/268 - Loss:  5.115, Seconds: 4.45
Average loss for this update: 5.176
New Record!
Epoch   2/100 Batch   20/268 - Loss:  4.826, Seconds: 3.03
Epoch   2/100 Batch   40/268 - 

In [ ]:
checkpoint = "/content/drive/Model 1/Model 300/best_model.ckpt" 

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    names = []
    [names.append(n.name) for n in loaded_graph.as_graph_def().node]
names

INFO:tensorflow:Restoring parameters from /content/drive/Model 1/Model 300/best_model.ckpt


['input',
 'targets',
 'learning_rate',
 'keep_prob',
 'summary_length',
 'Const',
 'max_dec_len',
 'text_length',
 'ReverseV2/axis',
 'ReverseV2',
 'embedding_lookup/params_0',
 'embedding_lookup/axis',
 'embedding_lookup',
 'embedding_lookup/Identity',
 'encoder_0/DropoutWrapperInit/Const',
 'encoder_0/DropoutWrapperInit/Const_1',
 'encoder_0/DropoutWrapperInit_1/Const',
 'encoder_0/DropoutWrapperInit_1/Const_1',
 'encoder_0/bidirectional_rnn/fw/fw/Rank',
 'encoder_0/bidirectional_rnn/fw/fw/range/start',
 'encoder_0/bidirectional_rnn/fw/fw/range/delta',
 'encoder_0/bidirectional_rnn/fw/fw/range',
 'encoder_0/bidirectional_rnn/fw/fw/concat/values_0',
 'encoder_0/bidirectional_rnn/fw/fw/concat/axis',
 'encoder_0/bidirectional_rnn/fw/fw/concat',
 'encoder_0/bidirectional_rnn/fw/fw/transpose',
 'encoder_0/bidirectional_rnn/fw/fw/sequence_length',
 'encoder_0/bidirectional_rnn/fw/fw/Shape',
 'encoder_0/bidirectional_rnn/fw/fw/strided_slice/stack',
 'encoder_0/bidirectional_rnn/fw/fw/strid

## Making Our Own Summaries

To see the quality of the summaries that this model can generate, you can either create your own review, or use a review from the dataset. You can set the length of the summary to a fixed value, or use a random value like I have here.

In [ ]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [ ]:
input_sentence = "office depot is branching into coworking space opening its first workonomy hub in its los gatos california store the firm s workonomy concept is targeted at smes a range of services are expected to roll out to 1 000 stores by end august with 141 stores lined up to host tech support kiosks while rival staples charges 130 month membership to tap into its coworking offering which launched in partnership with workbar in 2016 office depot s prices range from 40 day to 750 month for a private office\
                office depot stores to host its workonomy coworking spaces."
text = text_to_seq(input_sentence)
random = np.random.randint(0,len(clean_texts))
input_sentence = clean_texts[random]
text = text_to_seq(clean_texts[random])
checkpoint = "/content/drive/Model 1/Model 300/best_model.ckpt" 



loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
pad = vocab_to_int["<PAD>"] 

print('Original Text:', news.Text[random])
print('Original summary:', news.Summary[random])#clean_summaries[random]

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from /content/drive/Model 1/Model 300/best_model.ckpt
Original Text: VTB, Sberbank and the Bank of Moscow have announced they became the target of an extensive denial-of-service (DDoS) attack last autumn. The extortionists demanded 50 Bitcoins in return for stopping the attacks at a total value of around $11,500. Whilst Russia's three largest banks did not agree to pay the hackers, the banks have not experienced large-scale DDoS attacks since and did not suffer any damage as a result of the attack.

Original summary: Russia’s top 3 banks were target of world’s largest DDoS attack


Text
  Word Ids:    [4756, 1938, 211, 5877, 1641, 20180, 982, 14558, 16744, 997, 10783, 2025, 3793, 2260, 31321, 7206, 271, 2155, 793, 6907, 26, 1470, 936, 2100, 1002, 818, 9901, 2993, 265, 671, 1440, 2622, 207, 2126, 1440, 3260, 1406, 1232, 10783, 26, 3484, 3736, 4485, 4298, 2025]
  Input Words: vtb sberbank bank moscow announced became target extensive denial service dd

## Bleu Test

In [ ]:
import nltk

hypothesis =  ['struggling', 'latin', 'america', 'hit','its','multiple','risk' ]     
reference = ['Struggling' ,'Latin', 'America', 'multiple' , 'further' , 'shocks'  ]  
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)

0.6147881529512643


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
reference = ['Facebook', 'stays', 'popular', 'with','publishers','despite','poor', 'transparency' ,'at', 'risk' ]     
candidate = ['facebook' ,'depot', 'flexible', 'office' , 'working'  ]       
score = sentence_bleu(reference, candidate)
print(score)

0


In [ ]:
!pip install sumeval
!python -m spacy download en

     |████████████████████████████████| 81kB 5.5MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
  Created wheel for sumeval: filename=sumeval-0.2.2-cp36-none-any.whl size=54535 sha256=37da8e3b1aa49a20c49700dd5eb7738e368d2302c456daf34a14034fc31c082a
  Stored in directory: /root/.cache/pip/wheels/7b/6f/57/19ceecab21445c88f3c565735fa1887b4cd18d340c972eb445
Successfully built sumeval
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
#https://github.com/chakki-works/sumeval
#https://github.com/Tian312/awesome-text-summarization

from sumeval.metrics.rouge import RougeCalculator

refrence_summary = "Struggling Latin America faces multiple further shocks"
model_summary = "struggling latin america hit multiple risk"

rouge = RougeCalculator(stopwords=True, lang="en")

rouge_1 = rouge.rouge_n(
            summary=model_summary,
            references=refrence_summary,
            n=1)

rouge_2 = rouge.rouge_n(
            summary=model_summary,
            references=[refrence_summary],
            n=2)

rouge_l = rouge.rouge_l(
            summary=model_summary,
            references=[refrence_summary])

# You need spaCy to calculate ROUGE-BE

rouge_be = rouge.rouge_be(
            summary=model_summary,
            references=[refrence_summary])

print("ROUGE-1: {}, ROUGE-2: {}, ROUGE-L: {}, ROUGE-BE: {}".format(
    rouge_1, rouge_2, rouge_l, rouge_be
).replace(", ", "\n"))

b.multiple=(amod)=>risk
a.risk=(dobj)=>hit
<BasicElement: risk-[dobj]->hit>
b.further=(amod)=>shocks
a.shocks=(dobj)=>faces
<BasicElement: shocks-[dobj]->face>
ROUGE-1: 0.6666666666666666
ROUGE-2: 0.4000000000000001
ROUGE-L: 0.6666666666666666
ROUGE-BE: 0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

sentence = ("""office depot is branching into coworking space opening its first workonomy hub in its los gatos california store the firm s workonomy concept is targeted at smes a range of services are expected to roll out to 1 000 stores by end august with 141 stores lined up to host tech support kiosks while rival staples charges 130 month membership to tap into its coworking offering which launched in partnership with workbar in 2016 office depot s prices range from 40 day to 750 month for a private office.
""","")
model = ("office depot stores to host its workonomy coworking spaces","")
summary = ("office supply centre to expand its","")

tfidf_matrix_sentence = tfidf_vectorizer.fit_transform(sentence) 
tfidf_matrix_model = tfidf_vectorizer.transform(model) 
tfidf_matrix_summary = tfidf_vectorizer.transform(summary) 


cosine = cosine_similarity(tfidf_matrix_sentence, tfidf_matrix_model)
print(cosine)

cosine = cosine_similarity(tfidf_matrix_sentence, tfidf_matrix_summary)
print(cosine)


[[0.5976143 0.       ]
 [0.        0.       ]]
[[0.53674504 0.        ]
 [0.         0.        ]]


## Summary